# 03 — Resultados E0: consolidación, deltas y plots

Este notebook lee los outputs de **E0** y genera:
- Tabla consolidada (global) por modelo y feature set
- Deltas vs baseline (FS0)
- Figuras (barras) para incluir en la memoria

Requiere haber ejecutado **02_run_E0_ablation_training.ipynb** (para crear `outputs/E0_ablation/summary_metrics.csv`).

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd

# Ensure PROJECT_ROOT is on sys.path so `import src.*` works reliably
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR
if (PROJECT_ROOT / 'src').exists() is False and (PROJECT_ROOT.parent / 'src').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.e0_ablation_utils import get_project_paths

paths = get_project_paths(project_root=PROJECT_ROOT, output_dir='outputs/E0_ablation')
OUTPUT_DIR = paths.output_dir

summary_path = OUTPUT_DIR / 'summary_metrics.csv'
print('OUTPUT_DIR:', OUTPUT_DIR)
print('summary_path:', summary_path, 'exists=', summary_path.exists())

In [ ]:
if not summary_path.exists():
    raise FileNotFoundError(f'Missing {summary_path}. Run notebook 02 first.')

summary = pd.read_csv(summary_path)
display(summary)
print('rows:', len(summary))

## Tabla pivot (métricas globales)
Pivot por modelo y feature set para inspección rápida.

In [ ]:
metrics = ['MAE','RMSE','sMAPE']
pivot = summary.pivot_table(index='model', columns='feature_set', values=metrics)
display(pivot)

## Deltas vs FS0
Calculamos $\Delta$ por métrica y modelo, donde $\Delta = metric(FSx) - metric(FS0)$.
- Valores negativos en MAE/RMSE/sMAPE implican mejora.

In [ ]:
base = summary[summary['feature_set'] == 'FS0'][['model'] + metrics].set_index('model')

deltas = summary.copy()
for m in metrics:
    deltas[m] = deltas.apply(lambda r: r[m] - base.loc[r['model'], m], axis=1)

deltas = deltas.sort_values(['model','feature_set']).reset_index(drop=True)
display(deltas)

delta_path = OUTPUT_DIR / 'summary_deltas_vs_FS0.csv'
deltas.to_csv(delta_path, index=False)
print('Saved:', delta_path)

## Plots
Generamos gráficos de barras para cada métrica y los guardamos en `outputs/E0_ablation/figures/`.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig_dir = OUTPUT_DIR / 'figures'
fig_dir.mkdir(parents=True, exist_ok=True)

order_fs = ['FS0','FS1','FS2']

for m in metrics:
    plt.figure(figsize=(8, 4))
    sns.barplot(data=summary, x='feature_set', y=m, hue='model', order=order_fs)
    plt.title(f'E0 Ablation — {m}')
    plt.xlabel('Feature set')
    plt.ylabel(m)
    plt.tight_layout()
    out = fig_dir / f'e0_ablation_{m}.png'
    plt.savefig(out, dpi=160)
    plt.show()
    plt.close()
    print('Saved:', out)

## Export rápido para LaTeX (opcional)
Genera una tabla `.tex` con `pandas.to_latex()` basada en `summary_metrics.csv`.

Si prefieres una tabla con formato específico de la plantilla, usa `[COMPLETAR: ...]` y la adaptamos.

In [ ]:
tex_dir = OUTPUT_DIR / 'tables'
tex_dir.mkdir(parents=True, exist_ok=True)

tbl = summary.sort_values(['model','feature_set'])[['model','feature_set'] + metrics]
tex_path = tex_dir / 'e0_ablation_summary.tex'
tex_path.write_text(tbl.to_latex(index=False, float_format='%.4f'), encoding='utf-8')
print('Saved:', tex_path)